<a href="https://colab.research.google.com/github/franciscogarate/utils_ucm/blob/master/notebooks/ClimateRisk_2_Parques_Bomberos_GoogleV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Geoposicionar usando geopy (GoogleV3 - OpenStreetMap) y graficar con Folium

Datos: https://datos.madrid.es/sites/v/index.jsp?vgnextoid=fa3881ac7ad6b410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD

Los datos ya incluyen la geoposición (coordenadas utm y tambien *lon* y *lat*). No obstante, hacemos como sino tuvieramos estas coordenadas y las buscamos con la librería de código abierto de OpenStreetMap o API de Google.

In [9]:
!git clone https://github.com/franciscogarate/utils_ucm

Cloning into 'utils_ucm'...
remote: Enumerating objects: 243, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 243 (delta 0), reused 0 (delta 0), pack-reused 239 (from 1)
Receiving objects: 100% (243/243), 1.75 MiB | 25.97 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [3]:
import pandas as pd
import numpy as np
from geopy.geocoders import GoogleV3

In [6]:
from google.colab import userdata
api_key_googlev3 = userdata.get('api_key_google_v3')

In [7]:
# Pruebo que funcione
from geopy.geocoders import GoogleV3
geolocator = GoogleV3(api_key=api_key_googlev3)
location = geolocator.geocode("175 5th Avenue NYC")
print((location.latitude, location.longitude))

(40.7413871, -73.9893731)


In [8]:
# Creo mi propia función:
def direcc(x):
    try:
        location = geolocator.geocode(x)
        return (location.latitude, location.longitude)
    except:
        return (0, 0)

In [10]:
df = pd.read_csv('./utils_ucm/data/211642-1-bomberos-parques.csv', sep=';', usecols=(9,10,12))
df['Direccion'] = df.apply(lambda x: str(x['CLASE-VIAL'] + ' ' + x['NOMBRE-VIA']) + ', ' + str(x.NUM) + ', Madrid, Spain', axis=1)
df['coord'] = df['Direccion'].apply(lambda x: direcc(x))
df

,NOMBRE-VIA,CLASE-VIAL,NUM,Direccion,coord
0,SANTA ENGRACIA,CALLE,118,"CALLE SANTA ENGRACIA, 118, Madrid, Spain","(40.4401957, -3.7007281)"
1,RUFINO BLANCO,CALLE,2,"CALLE RUFINO BLANCO, 2, Madrid, Spain","(40.4280535, -3.6667525)"
2,SEGOVIA,RONDA,95,"RONDA SEGOVIA, 95, Madrid, Spain","(40.4065806, -3.7107557)"
3,EMILIA,CALLE,40,"CALLE EMILIA, 40, Madrid, Spain","(40.3450356, -3.713301)"
4,SANTA MARIA DE LA CABEZA,PASEO,95,"PASEO SANTA MARIA DE LA CABEZA, 95, Madrid, Spain","(40.3922304, -3.7066515)"
5,SAN BERNARDO,CALLE,68,"CALLE SAN BERNARDO, 68, Madrid, Spain","(40.426741, -3.7065449)"
6,HELLIN,AVENIDA,52,"AVENIDA HELLIN, 52, Madrid, Spain","(40.4284002, -3.6155508)"
7,PIO FELIPE,CALLE,24,"CALLE PIO FELIPE, 24, Madrid, Spain","(40.3946351, -3.653046299999999)"
8,MONFORTE DE LEMOS,AVENIDA,20,"AVENIDA MONFORTE DE LEMOS, 20, Madrid, Spain","(40.4800946, -3.6996555)"
9,SAN NORBERTO,CALLE,54,"CALLE SAN NORBERTO, 54, Madrid, Spain","(40.3399108, -3.7069834)"


In [11]:
#Guardo las coordenadas en un array:
coord_riesgos = np.asarray(df.coord)
coord_riesgos

array([(40.4401957, -3.7007281), (40.4280535, -3.6667525),
       (40.4065806, -3.7107557), (40.3450356, -3.713301),
       (40.3922304, -3.7066515), (40.426741, -3.7065449),
       (40.4284002, -3.6155508), (40.3946351, -3.653046299999999),
       (40.4800946, -3.6996555), (40.3399108, -3.7069834),
       (40.4742888, -3.6650986), (40.3878369, -3.7649168),
       (40.3911576, -3.5896932)], dtype=object)

## Gráfico con Folium

In [12]:
from folium import Map, plugins
from folium.plugins import HeatMap

Centro el mapa en Madrid:

In [13]:
map_madrid = Map(location=[40.41686, -3.7041056], zoom_start = 12)

In [14]:
coord_riesgos = np.asarray(df.coord)
HeatMap(coord_riesgos, radius=20).add_to(map_madrid)
map_madrid

### Calculo del punto de mayor concentracion en un radio de 500 metros con **scipy.spatial**

In [15]:
from scipy.spatial.distance import cdist

puntos = list(coord_riesgos)

# Calcular la matriz de distancias entre los puntos
distancias = cdist(puntos, puntos, 'euclidean')
distancias

array([[0.        , 0.03608011, 0.03507888, 0.0959871 , 0.04832967,
        0.01465824, 0.08599015, 0.06594939, 0.03991331, 0.1004798 ,
        0.04931329, 0.08283498, 0.12138156],
       [0.03608011, 0.        , 0.04896291, 0.09517738, 0.05362112,
        0.03981404, 0.05120287, 0.03611993, 0.06157015, 0.09688994,
        0.04626487, 0.10608301, 0.08543678],
       [0.03507888, 0.04896291, 0.        , 0.06159761, 0.01492557,
        0.02059545, 0.09767327, 0.05893276, 0.07434731, 0.06677644,
        0.08166377, 0.05731275, 0.12204097],
       [0.0959871 , 0.09517738, 0.06159761, 0.        , 0.04766094,
        0.08198425, 0.12847085, 0.07804319, 0.13574658, 0.00813484,
        0.13794876, 0.06705328, 0.13193228],
       [0.04832967, 0.05362112, 0.01492557, 0.04766094, 0.        ,
        0.03451076, 0.09801832, 0.05365911, 0.08814228, 0.05232065,
        0.09197948, 0.05843071, 0.11696322],
       [0.01465824, 0.03981404, 0.02059545, 0.08198425, 0.03451076,
        0.        , 0.09100

Calculo la cantidad de puntos dentro del radio de 500 metros para cada punto

In [16]:
puntos_concentrados = []

for i, punto in enumerate(puntos):
    puntos_cercanos = distancias[i] < .05   # Puntos dentro del radio
    concentracion = sum(puntos_cercanos) - 1  # Restar 1 para excluir el propio punto
    puntos_concentrados.append((punto, concentracion))

puntos_concentrados

[((40.4401957, -3.7007281), np.int64(6)),
 ((40.4280535, -3.6667525), np.int64(5)),
 ((40.4065806, -3.7107557), np.int64(4)),
 ((40.3450356, -3.713301), np.int64(2)),
 ((40.3922304, -3.7066515), np.int64(4)),
 ((40.426741, -3.7065449), np.int64(4)),
 ((40.4284002, -3.6155508), np.int64(1)),
 ((40.3946351, -3.653046299999999), np.int64(1)),
 ((40.4800946, -3.6996555), np.int64(2)),
 ((40.3399108, -3.7069834), np.int64(1)),
 ((40.4742888, -3.6650986), np.int64(3)),
 ((40.3878369, -3.7649168), np.int64(0)),
 ((40.3911576, -3.5896932), np.int64(1))]

Ordeno la lista de puntos por concentración de mayor a menor e imprimo la lista de puntos con mayor concentración

In [17]:
puntos_concentrados.sort(key=lambda x: x[1], reverse=True)

for punto, concentracion in puntos_concentrados:
    print(f"Punto: {punto}, Concentracion: {concentracion}")

Punto: (40.4401957, -3.7007281), Concentracion: 6
Punto: (40.4280535, -3.6667525), Concentracion: 5
Punto: (40.4065806, -3.7107557), Concentracion: 4
Punto: (40.3922304, -3.7066515), Concentracion: 4
Punto: (40.426741, -3.7065449), Concentracion: 4
Punto: (40.4742888, -3.6650986), Concentracion: 3
Punto: (40.3450356, -3.713301), Concentracion: 2
Punto: (40.4800946, -3.6996555), Concentracion: 2
Punto: (40.4284002, -3.6155508), Concentracion: 1
Punto: (40.3946351, -3.653046299999999), Concentracion: 1
Punto: (40.3399108, -3.7069834), Concentracion: 1
Punto: (40.3911576, -3.5896932), Concentracion: 1
Punto: (40.3878369, -3.7649168), Concentracion: 0
